In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import cv2
import os
import cvlib as cv
import numpy as np

In [2]:
model = load_model('gender_detection.model')

In [3]:
webcam = cv2.VideoCapture(0)

In [4]:
classes = ['man','women']

In [5]:
# loop through frames
while webcam.isOpened():

    # read frame from webcam 
    status, frame = webcam.read()

    # apply face detection
    face, confidence = cv.detect_face(frame)
    
    
    # loop through detected face
    for idx, f in enumerate(face):
        
        #corner points
        (startx,starty) = f[0], f[1]
        (endx,endy) = f[2],f[3]
        
        #draw rectangle over frame
        cv2.rectangle(frame, (startx,starty),(endx,endy),(0,255,0),2)
        
        #crop the detected face region
        face_crop = np.copy(frame[starty:endy, startx:endx])
        
        if (face_crop.shape[0])<10 or (face_crop.shape[1])<10:
            continue
            
         # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = model.predict(face_crop)[0] # model.predict return a 2D matrix, ex: [[9.9993384e-01 7.4850512e-05]]

        # get label with max accuracy
        idx = np.argmax(conf)
        label = classes[idx]

        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        Y = starty - 10 if starty - 10 > 10 else startY + 10

        # write label and confidence above face rectangle
        cv2.putText(frame, label, (startx, starty),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    # display output
    cv2.imshow("gender detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 55ms/step


TypeError: cannot unpack non-iterable NoneType object